In [42]:
import json
import requests
import datetime
import time
import os
import pandas as pd
from urllib.request import urlopen
import csv
from pprint import pprint
from datetime import datetime

In [43]:
# Fetching the information for the initial mapping of station_ids

def fetch_station_information():
    response_stations_info = urlopen('https://gbfs.divvybikes.com/gbfs/en/station_information.json')
    stations_info_json = json.loads(response_stations_info.read().decode("utf-8"))
    
    station_id_mapping = {}  # Create an empty dictionary to store the mapping
    
    # Create a mapping from station_id to an integer
    for index, station_info in enumerate(stations_info_json['data']['stations']):
        station_id = station_info['station_id']
        station_id_mapping[station_id] = index + 1  # Add 1 to start indexing from 1
    
    return station_id_mapping

In [44]:
# Storing the mapped data to CSV

def save_station_mapping_to_csv(station_id_mapping, filename='station_mapping.csv'):
    with open(filename, 'w', newline='') as csvfile:
        fieldnames = ['station_id', 'mapped_integer']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        writer.writeheader()
        for station_id, mapped_integer in station_id_mapping.items():
            writer.writerow({'station_id': station_id, 'mapped_integer': mapped_integer})


In [45]:
station_id_mapping = fetch_station_information()
save_station_mapping_to_csv(station_id_mapping)

In [46]:
# Function to load station ID mapping from CSV
def load_station_mapping_from_csv(filename='station_mapping.csv'):
    station_id_mapping = {}
    try:
        with open(filename, 'r') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                station_id_mapping[row['station_id']] = int(row['mapped_integer'])
    except FileNotFoundError:
        print(f"CSV file '{filename}' not found.")
    
    return station_id_mapping

# Function to update station information with mapping data
def update_station_info(station_info_json, station_id_mapping):
    # Create a set of valid station IDs
    valid_station_ids = set(station_id_mapping.keys())
    
    # Filter out stations that are not in the mapping
    station_info_json['data']['stations'] = [
        station for station in station_info_json['data']['stations'] if station['station_id'] in valid_station_ids
    ]
    
    # Update station IDs
    for station in station_info_json['data']['stations']:
        station_id = station['station_id']
        station['station_id'] = station_id_mapping.get(station_id, station_id)

# Function to update station status JSON with mapping data
def update_station_status(status_json, station_id_mapping):
    # Filter out stations that are not in the mapping
    status_json['data']['stations'] = [
        station for station in status_json['data']['stations'] if station['station_id'] in station_id_mapping
    ]
    
    # Update station IDs
    for station_status in status_json['data']['stations']:
        station_id = station_status['station_id']
        station_status['station_id'] = station_id_mapping.get(station_id, station_id)

In [47]:
def fetch_and_save_divvy_data():
    response_stations_info = urlopen('https://gbfs.divvybikes.com/gbfs/en/station_information.json')

    response = urlopen('https://gbfs.divvybikes.com/gbfs/en/station_status.json')

    # Extract the body of the reply
    response_body = response.read()
    response_stations_info = response_stations_info.read()

    # Decode the format in json format
    stations_json = json.loads(response_body.decode("utf-8"))
    stations_info_json = json.loads(response_stations_info.decode("utf-8"))
    
    # Update station information with mapping in-place
    station_id_mapping = load_station_mapping_from_csv()

    update_station_info(stations_info_json, station_id_mapping)

    update_station_status(stations_json, station_id_mapping)
     
    stations_dictionary = stations_json['data']
    stations_info_dictionary = stations_info_json['data']
    
    # load data file into a dataframe
    df__stations = pd.DataFrame(stations_dictionary['stations'])
    df__stations_info = pd.DataFrame(stations_info_dictionary['stations'])
    
    divvy_stations_realtime_status = pd.DataFrame(columns=[
                'altitude',
                'availableBikes',
                'availableDocks',
                'city',
                'id',
                'is_renting',
                'kioskType',
                'landMark',
                'lastCommunicationTime',
                'latitude',
                'location',
                'longitude',
                'postalCode',
                'stAddress1',
                'stAddress2',
                'stationName',
                'status',
                'statusKey',
                'statusValue',
                'testStation',
                'totalDocks' ])
    
    for x in range(len(df__stations.index)):
#         if not ((df__stations['station_id'].iloc[x]).isnumeric()) :
#             continue
        
        has_kiosk = df__stations_info['has_kiosk'][df__stations_info['station_id'] == df__stations['station_id'].iloc[x] ].values[0]
        
        row = [
            int(0),
            int(df__stations['num_bikes_available'].iloc[x]),
            int(df__stations['num_docks_available'].iloc[x]),
            'Chicago',
            df__stations['station_id'].iloc[x],
            bool(df__stations['is_renting'].iloc[x]),
            has_kiosk,
            'Chicago',
            datetime.strptime(time.ctime(int(df__stations['last_reported'].iloc[x])), "%a %b %d %H:%M:%S %Y").strftime("%Y-%m-%d %H:%M:%S"),
            float(df__stations_info['lat'][df__stations_info['station_id'] == df__stations['station_id'].iloc[x] ].values[0]),
            'Chicago',
            float(df__stations_info['lon'][df__stations_info['station_id'] == df__stations['station_id'].iloc[x] ].values[0]),
            60602,
            'Chicago',
            'Chicago',
            df__stations_info['name'][df__stations_info['station_id'] == df__stations['station_id'].iloc[x] ].values[0],
            'IN_SERVICE',
            1,
            'IN_SERVICE',
            False,
            int(df__stations_info['capacity'][df__stations_info['station_id'] == df__stations['station_id'].iloc[x] ].values[0])]
    
        divvy_stations_realtime_status.loc[len(divvy_stations_realtime_status)] = row

#         print("\n", row, "\n")
        
    divvy_stations_realtime_status['altitude'] = divvy_stations_realtime_status['altitude'].astype(float)
    divvy_stations_realtime_status['availableBikes'] = divvy_stations_realtime_status['availableBikes'].astype(int)
    divvy_stations_realtime_status['availableDocks'] = divvy_stations_realtime_status['availableDocks'].astype(int)
    divvy_stations_realtime_status['city'] = divvy_stations_realtime_status.city.astype(str)
    divvy_stations_realtime_status['id'] = divvy_stations_realtime_status['id'].astype('int64')
    divvy_stations_realtime_status['is_renting'] = divvy_stations_realtime_status['is_renting'].astype(bool)
    divvy_stations_realtime_status['kioskType'] = divvy_stations_realtime_status['kioskType'].astype(str)
    divvy_stations_realtime_status['landMark'] = divvy_stations_realtime_status['landMark'].astype(str)
    divvy_stations_realtime_status['lastCommunicationTime'] = divvy_stations_realtime_status['lastCommunicationTime'].astype(str)
    divvy_stations_realtime_status['latitude'] = divvy_stations_realtime_status['latitude'].astype(float)
    divvy_stations_realtime_status['location'] = divvy_stations_realtime_status['location'].astype(str)
    divvy_stations_realtime_status['longitude'] = divvy_stations_realtime_status['longitude'].astype(float)
    divvy_stations_realtime_status['postalCode'] = divvy_stations_realtime_status['postalCode'].astype(int)
    divvy_stations_realtime_status['stAddress1'] = divvy_stations_realtime_status['stAddress1'].astype(str)
    divvy_stations_realtime_status['stAddress2'] = divvy_stations_realtime_status['stAddress2'].astype(str)
    divvy_stations_realtime_status['stationName'] = divvy_stations_realtime_status['stationName'].astype(str)
    divvy_stations_realtime_status['status'] = divvy_stations_realtime_status['status'].astype(str)
    divvy_stations_realtime_status['statusKey'] = divvy_stations_realtime_status['statusKey'].astype(int)
    divvy_stations_realtime_status['statusValue'] = divvy_stations_realtime_status['statusValue'].astype(str)
    divvy_stations_realtime_status['testStation'] = divvy_stations_realtime_status['testStation'].astype(bool)
    divvy_stations_realtime_status['totalDocks'] = divvy_stations_realtime_status['totalDocks'].astype(int)
    
    # lets store our data subset into a file
    # This way you have a copy of data stored in a file 
    # that you could use in case you want to debug a problem on the orginal data received

    divvy_stations_realtime_status.to_csv('divvy_stations_status.csv',sep=',', encoding='utf-8', index=False)
        

In [48]:
fetch_and_save_divvy_data()

In [49]:
def fetchNearestDivvy(lat, long):
    # Load Divvy station data from CSV
    divvy_stations_realtime_status = pd.read_csv('divvy_stations_status.csv')

    # Calculate distance between given coordinates and each station
    divvy_stations_realtime_status['distance'] = ((lat - divvy_stations_realtime_status['latitude']) ** 2 + 
                                                  (long - divvy_stations_realtime_status['longitude']) ** 2) ** 0.5

    # Sort stations by distance and get the nearest one
    nearest_station = divvy_stations_realtime_status.sort_values(by='distance').iloc[0]

    # Return details of the nearest station
    return nearest_station

In [52]:
fetchNearestDivvy(41.848600, -87.628800)

altitude                                     0.0
availableBikes                                10
availableDocks                                 1
city                                     Chicago
id                                           429
is_renting                                  True
kioskType                                   True
landMark                                 Chicago
lastCommunicationTime        2024-04-14 15:27:56
latitude                               41.850084
location                                 Chicago
longitude                             -87.632141
postalCode                                 60602
stAddress1                               Chicago
stAddress2                               Chicago
stationName              Wentworth Ave & 24th St
status                                IN_SERVICE
statusKey                                      1
statusValue                           IN_SERVICE
testStation                                False
totalDocks          